In [95]:
import os
import pandas as pd

train_data_path ="~/aiffel/dktc/data/train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [96]:
print(train_data["conversation"][0])
print(len(train_data))

지금 너 스스로를 죽여달라고 애원하는 것인가?
 아닙니다. 죄송합니다.
 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해? 진짜 죽여버리고 싶게.
 정말 잘못했습니다.
 너가 선택해. 너가 죽을래 네 가족을 죽여줄까.
 죄송합니다. 정말 잘못했습니다.
 너에게는 선택권이 없어. 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야.
 선택 못하겠습니다. 한번만 도와주세요.
 그냥 다 죽여버려야겠군. 이의 없지?
 제발 도와주세요.
3950


1. **데이터 EDA와 데이터 전처리가 적절하게 이뤄졌는가?**
    * 결측치 제거
    * 중복데이터 제거
    * 불용어 제거 - 생성 모델이 아니기때문에 불용어를 제거해 중요한 단어만 넣는것이 중요하다고 생각
    * 특수문자제거 또는 space bar로 변환
    * 영어가 섞여있을 경우 소문자로 변환
    * 숫자 처리는 어떻게 해야하는가? 숫자가 섞인 욕을 하는 경우가 있을 수 잇음. -> 숫자 그대로를 유지하는게 나을지 아니면 숫자를 의미를 가진 단어로 변환을 할지.
    * 형태소 분석기를 사용하여 단어사전의 규모를 줄이는게 나을 수도 있지 않을까함. 그럴려면 맞춤법 검사기를 돌려야하나 생각이 들긴 함.
    * 토큰화와 패딩.
    * one hot encoding
    * 데이터를 분할
2. **Task에 알맞게 적절한 모델을 찾아보고 선정했는가?**
    * 임베딩 레이어를 통해 고정된 크기의 벡터로 변환
    * RNN: LSTM, GRU
        * 대화의 흐름을 이해하고 감정을 예측하는 데 효과적
        * LSTM과 GRU는 긴 시퀀스에서도 정보를 기억하고 사용하여 문장의 전후 맥락을 이해할 수 있음. 따라서 대화의 맥락을 이해하고 위협성을 판단하는 데 유리
        * * loss = Sparse Categorical Crossentropy (다중 클래스 분류 문제에서 주로 사용되기 때문)
    * 1D CNN
        * : CNN은 텍스트 데이터에서 지역 패턴(예: 특정 단어 조합이나 구문)을 감지하는 데 뛰어납니다. 이는 특정 위협적인 표현을 감지하는 데 유리
        * loss='categorical_crossentropy'
    * BERT 한국어 모델
        * BERT는 문장의 양방향 문맥을 이해
        * BERT는 주로 텍스트 분류, 문장 유사성 평가 등에 사용
        * loss='sparse_categorical_crossentropy'
    
3. **성능향상을 위해 논리적으로 접근했는가?**
4. **결과 도출을 위해 여러가지 시도를 진행했는가?**
5. **도출된 결론에 충분한 설득력이 있는가?**
6. **적절한 metric을 설정하고 그 사용 근거 및 결과를 분석하였는가?**
    * 정확도(Accuracy): 전체 데이터 중 맞게 분류된 데이터의 비율.
    * 정밀도(Precision): 실제로 위협이 아닌 대화를 위협으로 잘못 예측한 경우를 줄이기 위한 지표.
    * 재현율(Recall): 실제 위협 대화를 위협으로 잘 예측하는 비율.
    * Confusion Matrix: 모델이 예측한 클래스와 실제 클래스 간의 혼동을 시각적으로 표현하여 어떤 클래스에서 오류가 발생하는지 확인
7. **발표가 매끄럽게 진행되었고 발표시간을 준수하였는지?**

### 결측치 제거

In [97]:
print(train_data.isnull().sum())

idx             0
class           0
conversation    0
dtype: int64


In [98]:
print(train_data.isnull().sum()['class'])

0


### 중복제거

In [99]:
print(len(train_data))

3950


In [100]:
print('conversation열에서 중복을 배제한 유일한 샘플의 수 :', train_data['conversation'].nunique())

conversation열에서 중복을 배제한 유일한 샘플의 수 : 3846


In [101]:
print(train_data['conversation'].value_counts())

저기요. 잠깐만요\n매일 왜이러세요. 전 할말 없어요\n그 쪽이 좋아서요\n전 관심 없어요. 오지 마세요\n한 번만 만나주세요.\n제발 이러지 마세요. 전 남자친구 있어요\n결혼한 건 아니잖아요. 딱 한번만 데이트해요\n내가 왜 그쪽이랑 만나요?\n그 쪽이 내 스타일이에요\n난 아니거든요.                                                                                                                                                        2
야 석탄?\n 내가 왜 석탄이야?\n 얼굴이 까마니까 석탄이지.\n 그만해.\n 너 라이터 불붙이면 아주 잘 타겠다.\n 가 비켜.\n 아주 콧구멍 커지니까 이제 연탄이네.\n 비키라고.\n 야 연탄?\n 제발 좀 그만해. 나 정말 힘들어.                                                                                                                                                                                    2
진용아 내 내 가방 돌려줘\n아하하 진용아? 가방 돌려줄까?\n노 놀리지마\n노 놀리지마\n하아.\n한숨 쉰 거야?\n아 아 아아니야.\n아 아 아 아니야? 하하하하\n아아.\n왜? 기분 나빠?                                                                                                                                                                                                              2
김대리. 이번 주간 회의 발표 준비좀 해줘.\n저 말입니까.?\n응. 너가 해줘\n과장님 저번

In [102]:
print(train_data[train_data['conversation'] == "저기요. 잠깐만요\n매일 왜이러세요. 전 할말 없어요\n그 쪽이 좋아서요\n전 관심 없어요. 오지 마세요\n한 번만 만나주세요.\n제발 이러지 마세요. 전 남자친구 있어요\n결혼한 건 아니잖아요. 딱 한번만 데이트해요\n내가 왜 그쪽이랑 만나요?\n그 쪽이 내 스타일이에요\n난 아니거든요."])

     idx      class                                       conversation
262  262  기타 괴롭힘 대화  저기요. 잠깐만요\n매일 왜이러세요. 전 할말 없어요\n그 쪽이 좋아서요\n전 관심...
932  932  기타 괴롭힘 대화  저기요. 잠깐만요\n매일 왜이러세요. 전 할말 없어요\n그 쪽이 좋아서요\n전 관심...


In [103]:
print(train_data[train_data['conversation'] == '야 석탄?\n 내가 왜 석탄이야?\n 얼굴이 까마니까 석탄이지.\n 그만해.\n 너 라이터 불붙이면 아주 잘 타겠다.\n 가 비켜.\n 아주 콧구멍 커지니까 이제 연탄이네.\n 비키라고.\n 야 연탄?\n 제발 좀 그만해. 나 정말 힘들어.'])

       idx      class                                       conversation
782    782  기타 괴롭힘 대화  야 석탄?\n 내가 왜 석탄이야?\n 얼굴이 까마니까 석탄이지.\n 그만해.\n 너...
2656  2656  기타 괴롭힘 대화  야 석탄?\n 내가 왜 석탄이야?\n 얼굴이 까마니까 석탄이지.\n 그만해.\n 너...


In [104]:
# inplace=True 를 설정하면 DataFrame 타입 값을 return 하지 않고 data 내부를 직접적으로 바꿉니다
train_data.drop_duplicates(subset = ['conversation'], inplace=True)
print('전체 샘플수 :', (len(train_data)))

전체 샘플수 : 3846


### 맞춤법검사, 형태소분석, 불용어제거, 특수문제 제거 등

In [64]:
#불용어
stopwords = ['은','는','이','가','을','를','에','이가','이는']

In [41]:
from konlpy.tag import Okt

# Okt 형태소 분석기 객체 생성
okt = Okt()

def morph_analyze(text):
    # 텍스트에서 형태소 분석
    morphs = okt.morphs(text)
    return morphs

In [ ]:
# 부산대 맞춤법 분석기
from hanspell import spell_checker
import requests
def check_spelling(text):
    try:
        # 맞춤법 검사 요청
        result = spell_checker.check(text)
        
        # 응답 데이터 확인
        data = result.as_dict()

        # 'checked' 키 확인
        if 'checked' in data:
            return data['checked']
        else:
            print("Unexpected response format:", data)
            return text

    except KeyError as e:
        # KeyError 발생 시 응답 데이터를 출력하여 디버깅
        print(f"KeyError: {e}. The response format might have changed.")
        print("Response data:", json.dumps(result, ensure_ascii=False, indent=4))
        return text
    
    except Exception as e:
        # 기타 예외 처리
        print(f"An error occurred: {e}")
        return text


In [ ]:
# naver 맞춤법 검사기 사용해보기 -> local에서 사용한 뒤 csv파일로 만들어서 가져오기
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import json
from bs4 import BeautifulSoup
import time

def spelling_checker(text):
    # 맞춤법 검사기 페이지 찾아가기
    driver = webdriver.Chrome()#ChromeDriverManager().install()
    try:
        driver.get("http://www.naver.com")
        elem = driver.find_element("name","query")
        elem.send_keys("맞춤법 검사기")
        elem.send_keys(Keys.RETURN)
        # 검사기 페이지에서 맞춤법 검사를 원하는 텍스트 넣기
        elem = driver.find_element(By.CLASS_NAME,"txt_gray")
        time.sleep(1)
        elem.send_keys(text)
        # 맞춤법 검사 시작 버튼 누르기
        elem = driver.find_element(By.CLASS_NAME,"btn_check").click()
        time.sleep(1)

        # 맞춤법 검사 후 결과 가져오기
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        print(soup.select("p._result_text.stand_txt")[0].text)
        corrected_text = soup.select("p._result_text.stand_txt")[0].text
        return corrected_text
    
    finally:
        # 드라이버 종료
        driver.quit()

    
# # 예시 텍스트
# sample_text = "저는 오늘 그 시람을 만났읍니다. 그리고 그 사람은 매우 친절했읍니다."

# # 맞춤법 검사 적용
# corrected_text = spelling_checker(sample_text)

# print("Original Text:", sample_text)
# print("Corrected Text:", corrected_text)

In [91]:
import re

def preprocess_sentence(sentence):
    # 맞춤법 검사
    
    # 특정 단어는 @~@되어있기 때문에 처리필요. -> 고쳐야함
    sentence = re.sub(r'@[^@]+@', 'pronoun', sentence)
    # 특수문자 제거 (문장내의 특수 문자제거)
    sentence = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z?.!\s]", "", sentence)
    
    # 영어라면 소문자로 변환
    sentence = sentence.lower() # 텍스트 소문자화
    #형태소 분석
    sentence = morph_analyze(sentence)
    
    # 불용어 제거
    sen = []
    for word in sentence:
        if word in stopwords:
            continue
        sen.append(word)
       
    sentence = ' '.join(sen)

    return sentence

In [93]:
text = "이거 할인된다면서요\n check해 보겠습니다\n아까 저 사람#은 할인해 주던데\n네 저 손님은 고객님이십니다\n고객을 차별하는 거예요?\n저희는 @등급제@로 운영돼서 그런 겁니다\n점장 불러오세요\n손님 이건 점장님도 해결해 주시지 못합니다\n그니까 점장 불러와\n진정을 좀 해보세요"

text = preprocess_sentence(text)
print(text)

거 할인 된다면서요 
  check 해 보겠습니다 
 아까 저 사람 할인 해 주던데 
 네 저 손님 고객 님 이십 니 다 
 고객 차별 하는 거 예요 ? 
 저희 pronoun 로 운영 돼서 그런 겁니다 
 점 장 불러오세요 
 손님 이건 점장님 도 해결 해 주시 지 못 합니다 
 그니까 점 장 불러와 
 진정 좀 해보세요


In [92]:
text = "이거 할인된다면서요\n CHeck해 보겠습니다\n아까 저 사#람은 할인해 주던데\n네 저 손님은 고객님이십니다\n고객을 차별하는 거예요?\n저희는 @등급제@로 운영돼서 그런 겁니다\n점장 불러오세요\n손님 이건 점장님도 해결해 주시지 못합니다\n그니까 점장 불러와\n진정을 좀 해보세요"

text = preprocess_sentence(text)
print(text)

거 할인 된다면서요 
  check 해 보겠습니다 
 아까 저 사람 할인 해 주던데 
 네 저 손님 고객 님 이십 니 다 
 고객 차별 하는 거 예요 ? 
 저희 pronoun 로 운영 돼서 그런 겁니다 
 점 장 불러오세요 
 손님 이건 점장님 도 해결 해 주시 지 못 합니다 
 그니까 점 장 불러와 
 진정 좀 해보세요


In [120]:
temp = train_data['conversation'].map(lambda x: preprocess_sentence(x))

In [121]:
train_data['conversation'][:10]

0    지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1    길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2    너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3    어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4    저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
5    나 이틀뒤에 가나다 음식점 예약좀 해줘. 저녁7시로.\n가나다 음식점이요.?\n응....
6    35번 손님 아이스커피 두잔나왔습니다\n아이스커피? \n네 맛있게드세요\n저기요 아...
7    경비아저씨 내차에 경고장 붙였어요?\n내가 여기 몇년째 사는데 아직도 내차인줄 모르...
8    이거 할인 된다면서요\n 확인해보겠습니다\n 아까 저 시람은 할인 해주던데\n 네 ...
9    자네 세금계산서 보내준 거 메일이랑 수치가 다르던데?\n그게.\n얼마나 해먹었어? ...
Name: conversation, dtype: object

In [118]:
temp[:10]

0    지금 너 스스로 를 죽이다 달라 고 애원 하다 것 인가 ? \n  아니다 . 죄송하...
1    길동 경찰서 이다 . \n 9시 40분 마트 에 폭발물 을 설치 하다 . \n 네 ...
2    너 되게 귀엽다 알 지 ? 나 보다 작다 남자 는 첨 보다 . \n 그만하다 . 니...
3    어이 거기 \n 예 ?? \n 너 말 이야 너 . 이리 오라 고 \n 무슨 일 . ...
4    저기 요 혹시 날 이 너무 뜨겁다 ? 저희 회사 에서 이 선크림 팔다 하다 번 손등...
5    나 이틀 뒤 에 가나다 음식점 예약 좀 해주다 . 저녁 7시 로 . \n 가나다 음...
6    35 번 손님 아이스커피 두 잔나 오다 \n 아이스커피 ? 네 맛있다 들다 \n 저...
7    경비 아저씨 내 차 에 경고 장 붙이다 ? \n 내 가 여기 몇 년 째 살다 아직도...
8    이 거 할인 되다 \n  확인 해보다 \n  아깝다 저 시 람 은 할인 해주다 \n...
9    자네 세금 계산서 보내다 거 메일 이랑 수치 가 다르다 ? \n 그게 . \n 얼마...
Name: conversation, dtype: object

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Make Vocabulary - using subwordTextEncoder
def makeVocabulary(data):
    tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    data, target_vocab_size=2**13)
    
    return tokenizer

In [ ]:
def getToken(data):
    tokenizer = makeVocabulary(data)
    # 시작 토큰과 종료 토큰에 고유한 정수를 부여합니다.
    START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
    print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
    print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])
    # 시작 토큰과 종료 토큰을 고려하여 +2를 하여 단어장의 크기를 산정합니다.
    VOCAB_SIZE = tokenizer.vocab_size + 2
    print(VOCAB_SIZE)
    return tokenizer, START_TOKEN, END_TOKEN, VOCAB_SIZE

In [ ]:
# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
def tokenize_and_filter(inputs, outputs, MAX_LENGTH):
    tokenized_inputs, tokenized_outputs = [], []
    for (sentence1, sentence2) in zip(inputs, outputs):
        # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
        sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
        sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

        # 최대 길이 이하인 경우에만 데이터셋으로 허용
        if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
            tokenized_inputs.append(sentence1)
            tokenized_outputs.append(sentence2)
  # 최대 길이로 모든 데이터셋을 패딩
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
    return tokenized_inputs, tokenized_outputs

In [105]:
train_data['class'].unique()

array(['협박 대화', '기타 괴롭힘 대화', '갈취 대화', '직장 내 괴롭힘 대화'], dtype=object)

In [109]:
# class_list = {'협박 대화': 0, '갈취 대화': 1. '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
train_data.loc[train_data['class'] == '협박 대화', 'class'] = 0
train_data.loc[train_data['class'] == '갈취 대화', 'class'] = 1
train_data.loc[train_data['class'] == '직장 내 괴롭힘 대화', 'class'] = 2
train_data.loc[train_data['class'] == '기타 괴롭힘 대화', 'class'] = 3
train_data.loc[train_data['class'] == '일반 대화', 'class'] = 4

In [110]:
train_data

,idx,class,conversation
0,0,0,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,0,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,3,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,1,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,1,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
3945,3945,3,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...
3946,3946,1,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...
3947,3947,2,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...
3948,3948,1,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...


In [111]:
train_data.join(pd.get_dummies(train_data['class']) )

,idx,class,conversation,0,1,2,3
0,0,0,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,1,0,0,0
1,1,0,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,1,0,0,0
2,2,3,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,0,0,0,1
3,3,1,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,0,1,0,0
4,4,1,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,0,1,0,0
...,...,...,...,...,...,...,...
3945,3945,3,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...,0,0,0,1
3946,3946,1,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...,0,1,0,0
3947,3947,2,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...,0,0,1,0
3948,3948,1,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...,0,1,0,0


In [ ]:
#고른 데이터 사용을 위해 각 class에서 80%씩 train dataset에 넣고 20%는 validation dataset에 넣기
def makeDataset(data):
    # stratify : class가 균등하게 나눠지게 됨.train_test_split stratify
    # conversation - X_train, label - y_train
    data0 = train_data[train_data['협박 대화'] == 1]
    data1 = train_data[train_data['갈취 대화'] == 1]
    data2 = train_data[train_data['직장 내 괴롭힘 대화'] == 1]
    data3 = train_data[train_data['기타 괴롭힘 대화'] == 1]
    data4 = train_data[train_data['일반 대화'] == 1]
    
     # 각 데이터를 80%는 train, 20%는 validation으로 분할
    train_data1, val_data1 = train_test_split(data1, test_size=0.2, random_state=42)
    train_data2, val_data2 = train_test_split(data2, test_size=0.2, random_state=42)
    train_data3, val_data3 = train_test_split(data3, test_size=0.2, random_state=42)
    train_data4, val_data4 = train_test_split(data4, test_size=0.2, random_state=42)
    train_data5, val_data5 = train_test_split(data5, test_size=0.2, random_state=42)
    
    
    # 각 카테고리의 train 데이터를 하나로 합침
    X_train = pd.concat([train_data1, train_data2, train_data3, train_data4, train_data5])

    # 각 카테고리의 validation 데이터를 하나로 합침
    X_val = pd.concat([val_data1, val_data2, val_data3, val_data4, val_data5])

    return X_train, X_val 